# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [90]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df_o = pd.DataFrame(doc)
df_o.head(10)

,0
0,03/25/93 Total time of visit (in minutes):\n
1,6/18/85 Primary Care Doctor:\n
2,sshe plans to move as of 7/8/71 In-Home Servic...
3,7 on 9/27/75 Audit C Score Current:\n
4,2/6/96 sleep studyPain Treatment Pain Level (N...
5,.Per 7/06/79 Movement D/O note:\n
6,"4, 5/18/78 Patient's thoughts about current su..."
7,10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8,3/7/86 SOS-10 Total Score:\n
9,(4/10/71)Score-1Audit C Score Current:\n


In [91]:
import re

def func1(text):
    date1 = re.findall('(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z.]* (?:\d{1,2}(?:,|.) )?\d{2,4}', text)
    date2 = re.findall('(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z,]* (?:\d{1,2} )?\d{2,4}', text)
    date = None
    if date1 == [] and date2 == []:
        date = None
    elif date1 == []: 
        date = date2[0]
    else :
        date = date1[0]
    return date

def func2(text):
    date1 = re.findall('(?:\d{1,2}[/-])\d{1,2}[/-]\d{2,4}', text)
    date2 = re.findall('\d{1,2}[/-]\d{4}', text)
    if date1 == [] and date2 == []:
        date = None
    elif date1 != []:
        date = date1[0]
    else :
        date = date2[0]
    return date

def func3(text):
    date = re.findall('(?:19|20)\d{2}', text)
    if date == []:
        date = None
    else :
        date = date[0]
    return date

def func4(date):
    s1 = date.split('/')
    s2 = date.split('-')
    if len(s1) == 3 :
        day = int(s1[1])
        month = int(s1[0])
        year = int(s1[2])
    elif len(s1) == 2:
        day = 1
        month = int(s1[0])
        year = int(s1[1])
    elif len(s2) == 3:
        day = int(s2[1])
        month = int(s2[0])
        year = int(s2[2])
    elif len(date) == 4:
        day = 1
        month = 1
        year = int(date)
    return (day, month, year)


def func5(date):
    
    date = re.sub('[.,]', '', date)
    s = date.split(' ')
    
    month_dict = {'Jan' : 1, 'Feb' : 2, 'Mar' : 3, 'Apr' : 4, 'May' : 5, 'Jun' : 6, 'Jul' : 7, 'Aug' : 8, 'Sep' : 9,
                 'Oct' : 10, 'Nov' : 11, 'Dec' : 12}
    
    if len(s) == 3:
        if len(s[0]) in (1,2):
            day = int(s[0])
            month = month_dict[s[1][:3]]
        else :
            day = int(s[1])
            month = month_dict[s[0][:3]]
        year = int(s[2])
    elif len(s) == 2:
        day = 1
        month = month_dict[s[0][:3]]
        year = int(s[1])
        
    return (day, month, year)



In [92]:
def date_sorter():
    
    df = df_o.copy()
    df.columns = ['text']
    
    df['f1'] = df['text'].apply(lambda x : func1(x))
    df['f2'] = df['text'][(df['f1'].isnull())].apply(lambda x : func2(x))
    df['f3'] = df['text'][(df['f1'].isnull()) & (df['f2'].isnull())].apply(lambda x : func3(x))
    
    df['day'], df['month'], df['year'] = None,None,None
    
    cols = ['day', 'month', 'year']
    for n,col in enumerate(cols):
        df[col][(df['f1'].isnull() == False)] = df['f1'][(df['f1'].isnull() == False)].apply(lambda x : func5(x)[n])
        df[col][(df['f2'].isnull() == False)] = df['f2'][(df['f2'].isnull() == False)].apply(lambda x : func4(x)[n])
        df[col][(df['f3'].isnull() == False)] = df['f3'][(df['f3'].isnull() == False)].apply(lambda x : func4(x)[n])
    
    df['year'][(df['year'] < 100)] = df['year'][(df['year'] < 100)].apply(lambda x : x + 1900)
    
    sorted_index = df.sort_values(by=['year','month','day']).index
    
    return pd.Series(sorted_index)